# Orte für die Utrechter Priester für WIAG 2

[Zuordnung zu Orten in `place`](#Zuordnung-zu-bekannten-Orten-in-Tabelle-place)  
[Orte in `place` eintragen](#Orte-in-place-eintragen)

In [ ]:
wds_path = "C:\\Users\\georg\\Documents\\projekte\\WiagDataSetup.jl"

In [ ]:
using Pkg

In [ ]:
Pkg.activate(wds_path)

In [ ]:
using JSON, DataFrames, CSV

In [ ]:
using WiagDataSetup

In [ ]:
Wds = WiagDataSetup

In [ ]:
computer = :bkroege_goe
computer = :ghertko_kn
if computer == :bkroege_goe
    data_path = "../data"
    out_path = "../data_sql"
else
    data_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data"
    out_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"
end

In [49]:
file_name = joinpath(data_path, "AnonymousUser_ordinations_788_20220329_145955.json")

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data\\AnonymousUser_ordinations_788_20220329_145955.json"

In [50]:
d_place = JSON.parsefile(file_name);

In [ ]:
keys(d_place)

In [ ]:
features = d_place["features"];

In [ ]:
length(features)

In [ ]:
features[25]["@id"]

In [ ]:
features[25]["links"][1]["type"], features[25]["links"][1]["identifier"]

In [ ]:
get_whg_id(a) = replace(a, "http://whgazetteer.org/api/place/" => "")

In [ ]:
make_row(df::AbstractDataFrame) = Dict{Symbol, Any}(a => missing for a in Symbol.(names(df)))

In [51]:
df_place = DataFrame(
    id = String[],
    whg_id = String[],
    title = String[],
    link_type = Union{String, Missing}[],
    link_identifier = Union{String, Missing}[],
    type_label = Union{String, Missing}[],
    type_identifier = Union{String, Missing}[],
    coo_0 = Union{Float64, Missing}[],
    coo_1 = Union{Float64, Missing}[],
);

In [18]:
make_row(df_place)

Dict{Symbol, Any} with 9 entries:
  :coo_0           => missing
  :coo_1           => missing
  :link_identifier => missing
  :id              => missing
  :link_type       => missing
  :type_label      => missing
  :title           => missing
  :whg_id          => missing
  :type_identifier => missing

In [19]:
function fill_df_dict(df, features)
    i = 0
    n_max = Inf
    for f in features
        pr = make_row(df_place)
        pr[:id] = f["@id"]
        pr[:whg_id] = get_whg_id(f["properties"]["pid"])
        pr[:title] = f["properties"]["title"]
        for g in f["geometry"]["geometries"]
            if g["type"] == "Point"
                pr[:coo_0] = g["coordinates"][1]
                pr[:coo_1] = g["coordinates"][2]
            end
        end
        has_links = false
        for l in f["links"]
            has_links = true
            pr[:link_type] = l["type"]
            pr[:link_identifier] = l["identifier"]
            pr[:type_label] = missing
            pr[:type_identifier] = missing
            has_types = false
            for t in f["types"]
                pr[:type_label] = t["label"]
                pr[:type_identifier] = t["identifier"]
                push!(df_place, NamedTuple(pr))
            end
            if !has_types
                push!(df_place, NamedTuple(pr))
            end        
        end
        if !has_links
            push!(df_place, NamedTuple(pr))
        end    
        i += 1
        if i > n_max
            break
        end
    end
end    

fill_df_dict (generic function with 1 method)

In [52]:
fill_df_dict(df_place, features)

In [21]:
names(df_place)

9-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "link_type"
 "link_identifier"
 "type_label"
 "type_identifier"
 "coo_0"
 "coo_1"

In [22]:
file_name = joinpath(data_path, "priest_utrecht_places_a.csv")
CSV.write(file_name, df_place, delim=";")

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data\\priest_utrecht_places_a.csv"

In [23]:
size(df_place)

(2307, 9)

In [24]:
length(unique(df_place.id))

788

## Zuordnung zu bekannten Orten in Tabelle `place`
Die Koordinaten sind vermutlich in WGS84 angegeben, wie in `place` auch.

Die Zuordnung wird zunächst nicht verwendet. Die Orte werden unter einem eigenen Typ entsprechend ihrer Quelle in die Tabelle `place` aufgenommen.

In [ ]:
wds_path = "C:/Users/georg/Documents/projekte/WiagDataSetup.jl"

In [ ]:
cd(wds_path)

In [ ]:
using Pkg

In [ ]:
Pkg.activate(".")

In [ ]:
using WiagDataSetup

In [ ]:
Wds = WiagDataSetup

In [39]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [ ]:
using NearestNeighbors

In [ ]:
sql = "SELECT id, geonames_id, name, feature_code, latitude, longitude 
FROM place";
df_place_gn = Wds.sql_df(sql);

In [ ]:
p_data = Float64.(transpose(hcat(df_place_gn.longitude, df_place_gn.latitude)));

In [ ]:
tree = KDTree(p_data)

In [ ]:
df_place[50:55, [:id, :whg_id, :title, :coo_0, :coo_1]]

In [ ]:
p_sample = permutedims(hcat(df_place.coo_0, df_place.coo_1));

In [ ]:
range = 0.01
idx = inrange(tree, p_sample, range);

In [ ]:
size(idx)

In [ ]:
idx[100:105]

In [ ]:
count(x -> length(x) >= 2, idx), count(x -> length(x) == 1, idx), count(x -> length(x) == 0, idx)

In [ ]:
df_place[104, [:id, :whg_id, :title, :coo_0, :coo_1]]

In [ ]:
df_place_gn[[86491, 87462], :]

In [ ]:
df_place_gp = groupby(df_place, [:coo_0, :coo_1]);

Es gibt einen Ort bezogen auf die Koordinaten, der in der Quelle doppelt vorkommt.

In [ ]:
size(df_place_gp)

In [ ]:
for g in df_place_gp
    id = unique(g.id)
    if length(id) > 1
        println(id)
    end
end

Das scheint eine echte Dublette: Hasselt

Erstelle die Zuordnungstabelle `df_place_map`  
Füge `place_id` ein und verknüpfe mit `df_place_gn`

In [ ]:
df_place_map = similar(df_place, 0);

In [ ]:
df_place_map[!, :place_id] = Vector{Union{Int, Missing}}()
df_place_map[!, :n_match] = Vector{Int}()

In [ ]:
for (row, i_nn) in zip(eachrow(df_place), idx)
    row_data = collect(row)
    row_add = [missing, 0]
    n_match = length(i_nn)
    if n_match == 0
        push!(df_place_map, vcat(row_data, row_add))
    else
        for id in i_nn
            row_add = [df_place_gn[id, :id], n_match]
            push!(df_place_map, vcat(row_data, row_add))
        end
    end
end            

In [ ]:
size(df_place_map)

In [ ]:
df_place_map = leftjoin(df_place_map, df_place_gn, on = :place_id => :id, matchmissing = :notequal);

In [ ]:
size(df_place_map)

In [ ]:
sort!(df_place_map, :id);

In [ ]:
file_name = joinpath(data_path, "priest_utrecht_places_geonames.csv")
CSV.write(file_name, df_place_map, delim=";")

In [ ]:
df_place_map_gp = groupby(df_place_map, :n_match);

In [ ]:
typeof(df_place_map_gp[1])

In [ ]:
propertynames(df_place_map_gp)

In [ ]:
df_place_map_gp.keymap

In [ ]:
df_place_map_gp[3][7, [:name, :n_match]]

In [ ]:
combine(df_place_map_gp, :id => (x -> length(unique(x))) => :n_id)

In [ ]:
sum([90, 531, 137, 21, 6, 3])

## Orte in `place` eintragen
Quelle: `C:\Users\georg\Documents\projekte-doc\Utrecht-Priester\Ordinations of priests (identified place of origin).tsv`  
bzw. `data\AnonymousUser_ordinations_788_20220329_145955.json

siehe oben

In [ ]:
names(df_place)

In [ ]:
size(df_place)

Manche Einträge enthalten im Feld `link_identifier` einen Verweis auf eine GeoNames ID. Lies sie aus und schreibe sie in das Feld `geonames_id`.

In [ ]:
rgx = r"^gn:([0-9]+)"
function get_id_gn_maybe(s)
    if ismissing(s)
        return missing
    end
    rgm = match(rgx, s)
    if !isnothing(rgm)
        return(parse(Int,rgm[1]))
    else
        return missing
    end
end 

In [ ]:
transform!(df_place, :link_identifier => ByRow(get_id_gn_maybe) => :geonames_id);

In Bezug auf die folgenden Spalten ist jeweils nur ein Eintrag relevant.

In [ ]:
columns = [
    :id => :id_in_source,
    :geonames_id => :geonames_id,
    :title => :name,    
    :coo_1 => :latitude,
    :coo_0 => :longitude,
];

In [ ]:
df_place_ut = unique(select(df_place, columns));

In [ ]:
size(df_place_ut)

In [ ]:
df_place_ut_gn = subset(df_place_ut, :geonames_id => ByRow(!ismissing));

In [ ]:
size(df_place_ut_gn, 1), length(unique(df_place_ut_gn.id_in_source)) 

Einträge mit mehreren Verweisen auf GeoNames

In [ ]:
df_place_ut_gn_by_id = groupby(df_place_ut_gn, :id_in_source);

In [ ]:
df_tmp_count = combine(df_place_ut_gn_by_id, nrow => :n);

In [ ]:
df_place_ut_gn_n = leftjoin(df_place_ut_gn, df_tmp_count, on = :id_in_source);

In [ ]:
subset(df_place_ut_gn_n, :n => ByRow((x) -> (x > 1)))

In [ ]:
df_place_ut_gn_mg = antijoin(df_place_ut, df_place_ut_gn, on = :id_in_source);

In [ ]:
length(unique(df_place_ut_gn_mg.id_in_source))

In [ ]:
df_place_ut_out = vcat(df_place_ut_gn, df_place_ut_gn_mg);

In [ ]:
insertcols!(df_place_ut_out, :place_type_id => 2);

In [ ]:
table_name = "place"
file_name = joinpath(out_path, "$(table_name)_ut.sql")
Wds.insert_sql(file_name, table_name, df_place_ut_out);

In [ ]:
insertcols!(df_place_ut_gn_mg, :place_type_id => 2);

In [ ]:
out_path

```sql
source "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql/place_ut.sql";
```

### Ortsnamen eintragen

```sql
INSERT into place_label (place_id, geonames_id, label, lang, is_historic, is_geonames_name) 
(SELECT id as place_id, geonames_id, name as label, "nl", 1 as is_historic, 0 as is_geonames_name 
 FROM place
 WHERE place_type_id = 2);
```

### Geburtsorte zuordnen
Stapel hat für eine einen Teil der Geburtsorte Koordinaten angegeben. Sie werden genutzt für eine Zuordnung zu Orten in Tabelle `place`.

In [ ]:
file_name = joinpath(data_path, "..", "Ordinations of priests (identified place of origin).tsv")
df_priest_birthplace_in = CSV.read(file_name, DataFrame, delim = "\t");

In [ ]:
size(df_priest_birthplace_in)

In [ ]:
df_priest_birthplace_in[301:303, [:WKT_GEOM, :Priester_ID, :Toponiem, :Voornaam]]

In [ ]:
rgx = r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"

In [ ]:
rgm = match(rgx, "Point (6.0937644735630796 52.51259100584277917)")

In [ ]:
parse(Float64, rgm[1])

In [ ]:
function parse_wkt_point(wkt)
    rgx = r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"
    longitude = missing
    latitude = missing
    rgm = match(rgx, wkt)
    if !isnothing(rgm)
        if !isnothing(rgm[1])
            longitude = parse(Float64, rgm[1])
        end
        if !isnothing(rgm[2])
            latitude = parse(Float64, rgm[2])
        end
    end
    return longitude, latitude
end    

In [ ]:
columns = [
    :WKT_GEOM => :WKT_GEOM,
    :Priester_ID => :id_in_source,
    :Voornaam => :givenname,
    :Toponiem => :place_name_in,
]

In [ ]:
df_priest_bp = select(df_priest_birthplace_in, columns);

In [ ]:
transform!(df_priest_bp, :WKT_GEOM => ByRow(parse_wkt_point) => [:longitude, :latitude]);

In [ ]:
size(df_priest_bp), length(unique(df_priest_bp.id_in_source))

In [ ]:
df_priest_bp_n = groupby(df_priest_bp, :id_in_source)

In [ ]:
df_priest_bp_count = combine(df_priest_bp_n, nrow);

In [ ]:
df_priest_bp_mult = subset(df_priest_bp_count, :nrow => ByRow((x) -> (x > 1)));

In [ ]:
df_priest_bp_mult[51:55, :]

*2022-04-12*: Es ist nicht klar, wie die Zuordnung der Koordinaten zu interpretieren ist. Bei mehrfachen Zuordnungen gibt es einen Faktor. Die Summe der Faktoren ist 1.0.  
*2022-04-14*: Interpretiere den Faktor als die 'Wahrscheinlichkeit' für eine zutreffende Zuordnung.

### Verweis auf World Historical Gazeteer

In [25]:
authority_id_whg = 54

54

In [28]:
df_auth_whg = DataFrame()

""
""


In [29]:
insertcols!(df_auth_whg,
    :id => authority_id_whg,
    :url_formatter => "https://whgazetteer.org/places/{id}/detail",
    :url_name_formatter => "World Historical Gazetteer",
    :url_value_example => "6666407"
)

,id,url_formatter,url_name_formatter,url_value_example
,Int64,String,String,String
1,54,https://whgazetteer.org/places/{id}/detail,World Historical Gazetteer,6666407


In [30]:
table_name = "authority"
file_name = joinpath(out_path, "$(table_name)_whg.sql")
Wds.insert_sql(file_name, table_name, df_auth_whg);

Trage die Orte ein in `place_id_external`

In [32]:
df_place[51:55, [:id, :whg_id, :title, :coo_0, :coo_1]]

,id,whg_id,title,coo_0,coo_1
,String,String,String,Float64?,Float64?
1,brom_1040,6666419,Ghoes,3.88887,51.5063
2,brom_1040,6666419,Ghoes,3.88887,51.5063
3,brom_1040,6666419,Ghoes,3.88887,51.5063
4,brom_1041,6666420,Bat,4.21667,51.4
5,brom_1041,6666420,Bat,4.21667,51.4


In [53]:
df_place_u = unique(df_place, [:id, :whg_id, :title]);
rename!(df_place_u, [:id => :id_in_source]);

In [37]:
size(df_place_u)

(788, 9)

In [40]:
sql = "SELECT id, id_in_source, geonames_id, name from place where place_type_id = 2";
df_place_id = Wds.sql_df(sql);

In [43]:
df_place_id[31:34, :]

,id,id_in_source,geonames_id,name
,Int32,String,Int32?,String?
1,943712,brom_1250,2751592,Limmen
2,943713,brom_1268,2754692,Heemste
3,943714,brom_1268,2754691,Heemste
4,943715,brom_1276,2759631,Assendelff


In [41]:
size(df_place_id)

(796, 4)

In [54]:
df_place_u_id = innerjoin(df_place_u, df_place_id, on = :id_in_source);

In [56]:
size(df_place_u_id)

(796, 12)

In [62]:
columns = [
    :id => :place_id,
    :whg_id => :value,
    :geonames_id => :geonames_id
]

3-element Vector{Pair{Symbol, Symbol}}:
          :id => :place_id
      :whg_id => :value
 :geonames_id => :geonames_id

In [66]:
df_place_id_external = select(df_place_u_id, columns);

In [67]:
insertcols!(df_place_id_external,
    :authority_id => authority_id_whg
);

In [69]:
table_name = "place_id_external"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_place_id_external);